In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report 
from collections import Counter
from imblearn.over_sampling import SMOTE

# A. Read Dataset

In [2]:
df = pd.read_excel(r"your_file_path")
df.head(5)

,url,created_at,full_text,reply_count,retweet_count,user/advertiser_account_type,user/description,user/followers_count,user/friends_count,user/location,user/media_count,user/id,user/name,user/screen_name,user/url,user/verified
0,https://twitter.com/Cutty__Flam/status/1595649...,2022-11-24T05:24:20.000Z,@xa_adinn @RossaGhea @officialJKT48 ribet ga w...,1,0,promotable_user,"ARSENAL, LAZIO, T-ara, One Piece, and JKT48 lo...",57,254,Kemanggisan,1100,1246673035,Furanky,Cutty__Flam,NaN,False
1,https://twitter.com/jawapos/status/15962776895...,2022-11-25T23:00:33.000Z,"17 Desember, Pembelian BBM Subsidi di SPBU Waj...",0,0,promotable_user,"Akun Twitter resmi Jawa Pos Media | Jawa Pos, ...",512472,267,Jakarta,19383,2392460208,Jawa Pos,jawapos,https://t.co/zQIlbGPP2B,True
2,https://twitter.com/wahananewsdotco/status/159...,2022-11-26T01:24:33.000Z,"Data Konsumen Diambil Saat Isi BBM Subsidi, In...",0,2,promotable_user,Akun Resmi Media Wahana News - Inspirasi Konsu...,4837,1924,Jakarta,79,1124517245722349568,Wahana News Official,wahananewsdotco,https://t.co/CHwYCTxjr6,False
3,https://twitter.com/linkaja/status/15963681492...,2022-11-26T05:00:00.000Z,Isi BBM lebih banyak biar makin irit!\nBeli BB...,2,0,promotable_user,Layanan Customer Service LinkAja: Live Chat ht...,117575,13,NaN,10380,3195998996,LinkAja,linkaja,https://t.co/IfRujVkEqz,True
4,https://twitter.com/SyariahLinkAja/status/1596...,2022-11-27T11:00:02.000Z,"Isi BBM via MyPertamina, beli 20L dapat ekstra...",0,0,promotable_user,LinkAja Syariah adalah Aplikasi Dompet Elektro...,1906,4,NaN,3279,1297815347404263424,LinkAja Syariah,SyariahLinkAja,https://t.co/OgZKSEz68B,True


# B. Preprocessing
##### 1.Drop Duplicates 2.User Selection 3.Numbers Removal 4.Punctuation Removal  5.Whitespace Remover
##### 6.Stemming 7.Words Dictionary 8.Tokenizing 9.Stopwords Removal 10.Labeling

In [3]:
# fill the NaN 
df.fillna(0, inplace=True)
df.head(5)

,url,created_at,full_text,reply_count,retweet_count,user/advertiser_account_type,user/description,user/followers_count,user/friends_count,user/location,user/media_count,user/id,user/name,user/screen_name,user/url,user/verified
0,https://twitter.com/Cutty__Flam/status/1595649...,2022-11-24T05:24:20.000Z,@xa_adinn @RossaGhea @officialJKT48 ribet ga w...,1,0,promotable_user,"ARSENAL, LAZIO, T-ara, One Piece, and JKT48 lo...",57,254,Kemanggisan,1100,1246673035,Furanky,Cutty__Flam,0,False
1,https://twitter.com/jawapos/status/15962776895...,2022-11-25T23:00:33.000Z,"17 Desember, Pembelian BBM Subsidi di SPBU Waj...",0,0,promotable_user,"Akun Twitter resmi Jawa Pos Media | Jawa Pos, ...",512472,267,Jakarta,19383,2392460208,Jawa Pos,jawapos,https://t.co/zQIlbGPP2B,True
2,https://twitter.com/wahananewsdotco/status/159...,2022-11-26T01:24:33.000Z,"Data Konsumen Diambil Saat Isi BBM Subsidi, In...",0,2,promotable_user,Akun Resmi Media Wahana News - Inspirasi Konsu...,4837,1924,Jakarta,79,1124517245722349568,Wahana News Official,wahananewsdotco,https://t.co/CHwYCTxjr6,False
3,https://twitter.com/linkaja/status/15963681492...,2022-11-26T05:00:00.000Z,Isi BBM lebih banyak biar makin irit!\nBeli BB...,2,0,promotable_user,Layanan Customer Service LinkAja: Live Chat ht...,117575,13,0,10380,3195998996,LinkAja,linkaja,https://t.co/IfRujVkEqz,True
4,https://twitter.com/SyariahLinkAja/status/1596...,2022-11-27T11:00:02.000Z,"Isi BBM via MyPertamina, beli 20L dapat ekstra...",0,0,promotable_user,LinkAja Syariah adalah Aplikasi Dompet Elektro...,1906,4,0,3279,1297815347404263424,LinkAja Syariah,SyariahLinkAja,https://t.co/OgZKSEz68B,True


In [4]:
df.shape

(972, 16)

### 1. Droping Dataset

In [5]:
# drop duplicates
df.drop_duplicates(subset=['url'])
df.shape

(972, 16)

In [6]:
# user selection
df = df[df['user/name']!='MyPertamina']
df = df[df['user/name']!='LinkAja Syariah']
df = df[df['user/name']!='LinkAja']
df = df[df['user/name']!='Pertamina Papua Maluku']
df = df[df['user/name']!='Pertamax Series ID']
df = df[df['user/name']!='CNN Indonesia']
df = df[df['user/name']!='kumparan']
df = df[df['user/name']!='KOMPAS TV']
df = df[df['user/name']!='Kompas.com']
df = df[df['user/name']!='LIPUTAN6']
df = df[df['user/name']!='detikcom']
df = df[df['user/name']!='DetikFinance']
df = df[df['user/name']!='Detik jabar']
df = df[df['user/name']!='detik jatim']
df = df[df['user/name']!='detikoto']
df = df[df['user/name']!='detikinet']
df = df[df['user/name']!='Jawa Pos']
df = df[df['user/name']!='tempo.co']
df = df[df['user/name']!='brightgas']
df.shape

(747, 16)

### cut off time date

In [7]:
# fixing the date format
df['created_at'] = pd.to_datetime(df['created_at'])
df['created_at'] = df['created_at'].dt.date

# set the cut off point
start_date = pd.to_datetime('2022-5-25')
end_date = pd.to_datetime('2022-12-31')

# cutting the date
df = df[(df['created_at'] >= start_date) & (df['created_at'] < end_date)]

C:\Users\koman\AppData\Local\Temp\ipykernel_22960\248353203.py:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df = df[(df['created_at'] >= start_date) & (df['created_at'] < end_date)]


In [9]:
df.shape

(712, 16)

### creating id columns

In [10]:
# sorting dataframe
df = df.sort_values(by='created_at')
df = df.reset_index(drop=True)

In [11]:
# create id column
df['id'] = np.arange(1, len(df) +1)

# swaping column
def swap_columns(df, id, url):
    col_list = list(df.columns)
    x, y = col_list.index(id), col_list.index(url)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
df = swap_columns(df, 'url', 'id')

In [12]:
df_selected = df[['id', 'full_text', 'user/name', 'created_at']]

In [13]:
df_selected.head(5)

,id,full_text,user/name,created_at
0,1,"Setuju banget, BBM bersubsidi hanya diperuntuk...",Adam,2022-05-28
1,2,"Supaya Subsidi BBM Tepat Sasaran, Pembeli Pert...",Dina,2022-05-28
2,3,Nah solusi yang tepat!! \nSupaya Subsidi BBM T...,My time :),2022-05-29
3,4,"Supaya Subsidi BBM Tepat Sasaran, Pembeli Pert...",Riri! fb ya,2022-05-29
4,5,"Setuju banget, BBM bersubsidi hanya diperuntuk...",Lahrian Syahilla,2022-05-29


In [14]:
df_selected.shape

(712, 4)

### 2. Re-Formating Text

In [15]:
# remove numbers
index = df_selected.index.values

for i in index:
    df_nbremoved = (df_selected[['id', 'full_text']]).values
    df_nbremoved[i][1] = re.sub(r"\d+", "", df_nbremoved[i][1])
    df_selected.at[i, 'full_text'] = df_nbremoved[i][1]
    
df_selected.tail(5)

,id,full_text,user/name,created_at
707,708,"@MyPertaminaID pagi min, teman saya sudah daft...",ken,2022-12-02
708,709,@TimpalBali Aku pake myPertamina aja gadikais ...,kot,2022-12-02
709,710,"@MyPertaminaID min, saya sudah coba regist di ...",Ibrahim Maulana,2022-12-02
710,711,@MyPertaminaID \nJadi gimana kasus laporan ID ...,Irmat Samdo,2022-12-02
711,712,@s_arifin @MyPertaminaID dibales mypertamina dong,Muthiah Azzahra,2022-12-02


In [16]:
# remove punctuiation
index = df_selected.index.values

for i in index:
    df_pncremoved = (df_selected[['id', 'full_text']]).values
    df_pncremoved[i][1] = df_pncremoved[i][1].translate(str.maketrans("","",string.punctuation))
    df_selected.at[i, 'full_text'] = df_pncremoved[i][1]

df_selected.tail(5)

,id,full_text,user/name,created_at
707,708,MyPertaminaID pagi min teman saya sudah daftar...,ken,2022-12-02
708,709,TimpalBali Aku pake myPertamina aja gadikais m...,kot,2022-12-02
709,710,MyPertaminaID min saya sudah coba regist di pt...,Ibrahim Maulana,2022-12-02
710,711,MyPertaminaID \nJadi gimana kasus laporan ID ...,Irmat Samdo,2022-12-02
711,712,sarifin MyPertaminaID dibales mypertamina dong,Muthiah Azzahra,2022-12-02


### 3. Stemming

In [17]:
# stemming 
fact = StemmerFactory()
stemmer = fact.create_stemmer()

index = df_selected.index.values

for i in index:
    df_stem = (df_selected[['id', 'full_text']]).values
    df_stem[i][1] = stemmer.stem(df_stem[i][1])
    df_selected.at[i, 'full_text'] = df_stem[i][1]
    
df_selected.tail(5)

,id,full_text,user/name,created_at
707,708,mypertaminaid pagi min teman saya sudah daftar...,ken,2022-12-02
708,709,timpalbali aku pake mypertamina aja gadikais m...,kot,2022-12-02
709,710,mypertaminaid min saya sudah coba regist di pt...,Ibrahim Maulana,2022-12-02
710,711,mypertaminaid jadi gimana kasus lapor id kait ...,Irmat Samdo,2022-12-02
711,712,sarifin mypertaminaid dibales mypertamina dong,Muthiah Azzahra,2022-12-02


### 4. Words Dictionary

In [18]:
# words count
kamus = df_selected.full_text.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)

In [19]:
kamus_df = pd.DataFrame(kamus,columns=['total'])
text = kamus_df.index.values
kamus_df['word'] = text

# swaping column
def swap_columns(kamus_df, total, word):
    col_list = list(kamus_df.columns)
    x, y = col_list.index(total), col_list.index(word)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    kamus_df = kamus_df[col_list]
    return kamus_df
kamus_df = swap_columns(kamus_df, 'total', 'word')

# reseting index
kamus_df = kamus_df.reset_index(drop=True)

# modify column
kamus_df = kamus_df.astype({'total':'int'})

In [20]:
# filter words <= 10
kamus_kurangdari_10 = kamus_df[kamus_df['total']<=10]

### 5. Tokenizing

In [21]:
# tokenizing
df_token = df_selected['full_text'].apply(word_tokenize)
df_selected['full_text'] = df_token

df_selected.tail(5)

C:\Users\koman\AppData\Local\Temp\ipykernel_22960\3751136207.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['full_text'] = df_token


,id,full_text,user/name,created_at
707,708,"[mypertaminaid, pagi, min, teman, saya, sudah,...",ken,2022-12-02
708,709,"[timpalbali, aku, pake, mypertamina, aja, gadi...",kot,2022-12-02
709,710,"[mypertaminaid, min, saya, sudah, coba, regist...",Ibrahim Maulana,2022-12-02
710,711,"[mypertaminaid, jadi, gimana, kasus, lapor, id...",Irmat Samdo,2022-12-02
711,712,"[sarifin, mypertaminaid, dibales, mypertamina,...",Muthiah Azzahra,2022-12-02


### 6. Stopwords Removal

In [22]:
# ignore important words
df_ignored = pd.read_excel(r"your_file_path")
df_ignored = df_ignored['ignore'].tolist()
kamus_kurangdari_10['word'] = [' '.join([y for y in x.split() if y not in df_ignored])
                               for x in kamus_kurangdari_10['word']]

# adding stopwords
more_stopword_ind = pd.read_csv(r'your_file_path')
more_stopword = kamus_kurangdari_10['word'].tolist()

# merge stopwords
stopwords = more_stopword + more_stopword_ind['stop'].tolist()

# removing stopwords
index = df_selected.index.values
for i in index:
    df_stoprmv = (df_selected[['id', 'full_text']]).values
    df_stoprmv[i][1] = [x for x in df_stoprmv[i][1] if x not in stopwords]
    df_selected.at[i, 'full_text'] = df_stoprmv[i][1]

df_selected.tail(5)

C:\Users\koman\AppData\Local\Temp\ipykernel_22960\3176303690.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kamus_kurangdari_10['word'] = [' '.join([y for y in x.split() if y not in df_ignored])


,id,full_text,user/name,created_at
707,708,"[mypertaminaid, daftar, mypertamina, salah, so...",ken,2022-12-02
708,709,"[pake, mypertamina]",kot,2022-12-02
709,710,"[mypertaminaid, coba, mypertamina]",Ibrahim Maulana,2022-12-02
710,711,"[mypertaminaid, spbu, daftar, mypertamina, myp...",Irmat Samdo,2022-12-02
711,712,"[mypertaminaid, mypertamina]",Muthiah Azzahra,2022-12-02


### 7. Labeling

In [23]:
# labeling
listNegatif = pd.read_excel(r'your_file_path')
listNegatif = listNegatif['negatif'].tolist()
listPositif = pd.read_excel(r'your_file_path')
listPositif = listPositif['positif'].tolist()

df_selected["label"] = df_selected['full_text'].apply(lambda x: 'negatif' if any(i in x for i in listNegatif) 
                                                      else 'positif' 
                                                      if any(i in x for i in listPositif) else 'netral')

C:\Users\koman\AppData\Local\Temp\ipykernel_22960\758117357.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["label"] = df_selected['full_text'].apply(lambda x: 'negatif' if any(i in x for i in listNegatif)


In [24]:
df_positif = df_selected[df_selected['label']=='positif']
df_negatif = df_selected[df_selected['label']=='negatif']
df_netral = df_selected[df_selected['label']=='netral']

print(f'positif {df_positif.shape} record \n'
      f'negatif {df_negatif.shape} record \n'
      f'netral {df_netral.shape} record')

positif (251, 5) record 
negatif (189, 5) record 
netral (272, 5) record


In [25]:
data = df_selected[['id', 'full_text', 'user/name', 'created_at','label']]
data.tail(5)

,id,full_text,user/name,created_at,label
707,708,"[mypertaminaid, daftar, mypertamina, salah, so...",ken,2022-12-02,negatif
708,709,"[pake, mypertamina]",kot,2022-12-02,netral
709,710,"[mypertaminaid, coba, mypertamina]",Ibrahim Maulana,2022-12-02,netral
710,711,"[mypertaminaid, spbu, daftar, mypertamina, myp...",Irmat Samdo,2022-12-02,positif
711,712,"[mypertaminaid, mypertamina]",Muthiah Azzahra,2022-12-02,netral


In [26]:
data[['id', 'full_text', 'label']].head(5)

,id,full_text,label
0,1,"[tuju, bbm, subsidi, masyarakat, hak, subsidi,...",positif
1,2,"[subsidi, bbm, tepat, sasar, beli, pertalite, ...",positif
2,3,"[solusi, tepat, subsidi, bbm, tepat, sasar, be...",positif
3,4,"[subsidi, bbm, tepat, sasar, beli, pertalite, ...",positif
4,5,"[tuju, bbm, subsidi, masyarakat, hak, subsidi,...",positif


In [27]:
index = data.index.values

for i in index:
    increment = i+1
    seleksi = data[data['id']==increment]
    print(seleksi[['id','full_text']].values)


[[1
  list(['tuju', 'bbm', 'subsidi', 'masyarakat', 'hak', 'subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina'])]]
[[2
  list(['subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina', 'solusi', 'tepat', 'dukung'])]]
[[3
  list(['solusi', 'tepat', 'subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina'])]]
[[4
  list(['subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina', 'solusi', 'upaya', 'tepat'])]]
[[5
  list(['tuju', 'bbm', 'subsidi', 'masyarakat', 'hak', 'subsidi', 'bbm', 'tepat', 'sasar', 'beli', 'pertalite', 'atur', 'pakai', 'aplikasi', 'mypertamina'])]]
[[6
  list(['tepat', 'sasar', 'beli', 'pertalite', 'solar', 'pakai', 'mypertamina'])]]
[[7
  list(['perintah', 'aplikasi', 'mypertamina', 'salur', 'bahan', 'bakar', 'minyak', 'bbm', 'subsidi', 'solar', 'pertalite', 'tepat', 'sasar'])]]
[[8
  list(['b

[[480
  list(['pertamina', 'mypertamina', 'subsidi', 'tepat', 'bahan', 'bakar', 'pertamina', 'call', 'mypertamina'])]]
[[481
  list(['beli', 'pertalite', 'solar', 'pakai', 'mypertamina', 'data', 'kendara', 'data', 'bocor'])]]
[[482
  list(['mypertamina', 'batas', 'beli', 'pertalite', 'mypertamina', 'pertalite'])]]
[[483
  list(['masyarakat', 'beli', 'bbm', 'pakai', 'mypertamina', 'masyarakat', 'tuju', 'beli', 'bbm', 'pakai', 'mypertamina'])]]
[[484
  list(['subsidi', 'bbm', 'tepat', 'sasar', 'mypertamina', 'hubung', 'data', 'polisi', 'mypertamina', 'subsidi', 'bahan', 'bakar', 'minyak', 'bbm', 'tepat', 'sasar', 'batas'])]]
[[485 list([])]]
[[486 list(['pertalite', 'maaf', 'maaf', 'mypertamina'])]]
[[487
  list(['data', 'pertamina', 'data', 'pertalite', 'mypertamina', 'warga'])]]
[[488
  list(['beli', 'bbm', 'subsidi', 'atur', 'tepat', 'sasar', 'aplikasi', 'mypertamina', 'lengkap', 'spbu'])]]
[[489
  list(['salah', 'bbm', 'subsidi', 'tepat', 'sasar', 'salah', 'mypertamina'])]]
[[490
  l

# C. Spliting and TF-IDF

In [28]:
# split dataset 
X_train, X_test, y_train, y_test = train_test_split(  
                                                data['full_text'],  
                                                data['label'],   
                                                test_size=0.3,
                                                random_state=27)

In [29]:
print('y_test = ',y_test.shape)
print('X_test = ',X_test.shape)

print('y_train = ',y_train.shape)
print('X_train = ',X_train.shape)

y_test =  (214,)
X_test =  (214,)
y_train =  (498,)
X_train =  (498,)


In [30]:
# tf-idf
input_corrected_train = [" ".join(x) for x in X_train]
input_corrected_test = [" ".join(x) for x in X_test]


vectorizer = TfidfVectorizer()
train_tfidf = vectorizer.fit_transform(input_corrected_train)
test_tfidf = vectorizer.transform(input_corrected_test)

# convert to dataframe
train_tfidf = pd.DataFrame(train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
test_tfidf = pd.DataFrame(test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

In [31]:
train_tfidf

,aman,antre,antusiasme,aplikasi,atur,awas,bahan,bakar,bantu,batas,...,terima,tuju,uang,uji,untung,upaya,utk,via,wajib,warga
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.558914,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.278268,0.0,0.0,0.0,0.0,0.0,0.526786,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.432916,...,0.000000,0.370343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
494,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495,0.0,0.0,0.0,0.258598,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
test_tfidf

,aman,antre,antusiasme,aplikasi,atur,awas,bahan,bakar,bantu,batas,...,terima,tuju,uang,uji,untung,upaya,utk,via,wajib,warga
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.879551,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,0.0,0.0,0.0,0.212374,0.0,0.0,0.0,0.0,0.463138,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.169311,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.362988,0.367404
210,0.0,0.0,0.0,0.135637,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.565158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
211,0.0,0.0,0.0,0.195104,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
212,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


# D. SMOTE
###  Synthetic Minority Over-sampling Technique

In [33]:
# y_train before balancing
counter = Counter(y_train)
print(counter)

Counter({'netral': 188, 'positif': 180, 'negatif': 130})


In [34]:
# balancing training set
oversample = SMOTE()
x_sm, y_sm = oversample.fit_resample(train_tfidf, y_train)

In [35]:
# y_train after balancing
counter = Counter(y_sm)
print(counter)

Counter({'netral': 188, 'positif': 188, 'negatif': 188})


# E. NBC

In [36]:
# model with SMOTE
clfsm = MultinomialNB()
clfsm.fit(x_sm, y_sm)

# model without SMOTE
clf = MultinomialNB()
clf.fit(train_tfidf, y_train)

MultinomialNB()

In [37]:
# prediction without SMOTE
y_pred = clf.predict(test_tfidf)

# prediction with SMOTE
y_predsm = clfsm.predict(test_tfidf)

In [38]:
# prediction without smote
print("hasil prediksi tanpa mengimplementasikan smote: \n")
print(classification_report(y_test, y_pred))

hasil prediksi tanpa mengimplementasikan smote: 

              precision    recall  f1-score   support

     negatif       0.94      0.54      0.69        59
      netral       0.77      0.95      0.85        84
     positif       0.80      0.86      0.83        71

    accuracy                           0.81       214
   macro avg       0.84      0.78      0.79       214
weighted avg       0.83      0.81      0.80       214



In [39]:
# prediction with SMOTE
print("hasil prediksi dengan mengimplementasikan smote: \n")
print(classification_report(y_test, y_predsm))

hasil prediksi dengan mengimplementasikan smote: 

              precision    recall  f1-score   support

     negatif       0.83      0.73      0.77        59
      netral       0.82      0.88      0.85        84
     positif       0.86      0.87      0.87        71

    accuracy                           0.84       214
   macro avg       0.84      0.83      0.83       214
weighted avg       0.84      0.84      0.84       214

